Reliability Models for Facility Location:The Expected Failure Cost Case
===

Base on the following article:

*Snyder, L. V., & Daskin, M. S. (2005). Reliability models for facility location: the expected failure cost case. Transportation Science, 39(3), 400-416.*

Part III - The Reliability P-Median Problem
---

In [6]:
# Colecting the problem
import sys
sys.path.append('../../PythonLib')

from dataset.mongodb import MongoClient
# Solving the problem
from solvers.uflp import uflp
# Representing the network
from dataviz.network import Network

from sklearn.neighbors import DistanceMetric
from geopy.distance import great_circle

import scipy as sp
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First, we collect the dataset from our database using **MonClient**.

In [7]:
dataset = MongoClient(db = {
# MongoClient let us connect to the database and manipulate our dataset
    "mongo_host": "ns396089.ip-37-59-38.eu",
    "mongo_port": 32771,
    "mongo_db_name": "NETWORK-AND-DISCRETE-LOCATION"
}, q = {
    'metadata.author': 'Mark S. Daskin',
    'metadata.topic': 'NETWORK-AND-DISCRETE-LOCATION',
    'metadata.dataset': '49-nodes'
},f=None)

We can display the dataset using the method **get** which return a pandas Dataframe containing our data

In [8]:
df = dataset.get()
df.head()

,ID,LONGITUDE,LATITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,-121.467360,38.566850,29760021,369365,115800,< Sacramento CA >
1,2,-73.799017,42.665750,17990455,101082,101800,< Albany NY >
2,11,-84.422592,33.762900,6478216,394017,71200,< Atlanta GA >
3,12,-77.474584,37.531050,6187358,203056,66600,< Richmond VA >
4,13,-71.017892,42.336029,6016425,574283,161400,< Boston MA >


The reliable P-median objective aims to minimise of the weighted sum of the **operating cost** and the **expected failure cost**. Each facility as an expected **failure pobability**.